Code that can be used to estimate descending button auctions
(see working paper on Windstream and the RDOF)
Ignacio Nunez, PhD Candidate in Economics, UT-Austin, ijnunez@utexas.edu

In [45]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [62]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from statsmodels import api
from scipy import stats
from scipy import optimize
from scipy.optimize import minimize 
from scipy.optimize import show_options

# 1. Create sample with known parameters

In [47]:
#competitors parameters
mu_1=0; 
mu_1x=0.5;
mu_1y=2;
st_1=0.5;

#Windstream parameters
mu_2=0;
mu_2x=0.5;
mu_2d=2;
st_2=0.5;
kappa=0.5;

In [48]:
sample_size=500
# generate an independent variable 
x = np.linspace(-1, 2, sample_size)
# generate two normally distributed independent variables  residual
y = np.random.normal(0, 1, sample_size)
d = np.random.normal(0, 1, sample_size)

#Create sample
xi=np.random.normal(0, st_1, sample_size);
epsilon=xi*kappa+np.random.normal(0, st_2, sample_size);
X1 = mu_1+x*mu_1x+y*mu_1y+xi; #Competitors
X2 = mu_2+x*mu_2x+d*mu_2d+epsilon; #Windstream

df = pd.DataFrame({'x':x, 'y':y, 'd':d, 'Rivals':X1, 'Windstream':X2})
df.head()

,x,y,d,Rivals,Windstream
0,-1.000000,2.017288,0.149086,3.057529,-0.292670
1,-0.993988,-1.772899,-0.909086,-4.846528,-3.039842
2,-0.987976,1.386712,-0.185275,3.306495,0.441692
3,-0.981964,-0.850033,-0.403745,-2.245131,-0.930482
4,-0.975952,-0.183555,-0.118688,-0.537794,0.109605


In [100]:
df['Observed_LastBid']=0
df['Censored']=0
df.loc[df['Rivals']>=df['Windstream'],'Observed_LastBid']=df.loc[df['Rivals']>=df['Windstream'],'Rivals']
df.loc[df['Rivals']>=df['Windstream'],'Censored']=1
df.loc[df['Rivals']<df['Windstream'],'Observed_LastBid']=df.loc[df['Rivals']<df['Windstream'],'Windstream']
df.head()

Sample=df[['x','y','d','Observed_LastBid','Censored']]
Sample['constant']=1
Sample.head()

,x,y,d,Rivals,Windstream,Observed_LastBid,Censored
0,-1.000000,2.017288,0.149086,3.057529,-0.292670,3.057529,1
1,-0.993988,-1.772899,-0.909086,-4.846528,-3.039842,-3.039842,0
2,-0.987976,1.386712,-0.185275,3.306495,0.441692,3.306495,1
3,-0.981964,-0.850033,-0.403745,-2.245131,-0.930482,-0.930482,0
4,-0.975952,-0.183555,-0.118688,-0.537794,0.109605,0.109605,0


<ipython-input-100-28e2d640e433>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Sample['constant']=1


,x,y,d,Observed_LastBid,Censored,constant,cdf_aux
0,-1.000000,2.017288,0.149086,3.057529,1,1,NaN
1,-0.993988,-1.772899,-0.909086,-3.039842,0,1,NaN
2,-0.987976,1.386712,-0.185275,3.306495,1,1,NaN
3,-0.981964,-0.850033,-0.403745,-0.930482,0,1,NaN
4,-0.975952,-0.183555,-0.118688,0.109605,0,1,NaN


# 2. Generate simulation draws for SMLE estimation

In [101]:
sim_draws = np.random.normal(0, 1, 1000)
df_draws = pd.DataFrame({'draws':sim_draws})
df_draws['constant']=1
df_draws.head()

,draws,constant
0,-0.683938,1
1,1.431461,1
2,-0.137604,1
3,-1.454913,1
4,0.513520,1


# 3. Define Likelihood function

In [105]:
 # Likelihood function
def Likelihood(parameters):
    mu_1, mu_1x, mu_1y,st_1,mu_2,mu_2x,mu_2d,st_2,kappa= parameters
    st_1=np.abs(st_1+0.001)
    st_2=np.abs(st_2+0.001)

    aux=Sample.copy()
    aux['Windstream']=mu_2*aux['constant']+mu_2x*aux['x']+mu_2d*aux['d'] #mean utility Windstream   
    aux['Rivals']=mu_1*aux['constant']+mu_1x*aux['x']+mu_1y*aux['y'] #mean utility rivals

    aux_unc = aux.loc[aux['Censored']==0,:] 
    aux_cen = aux.loc[aux['Censored']==1,:] 

    LL1 = np.sum(stats.norm.logpdf(((aux_cen['Observed_LastBid']-aux_cen['Rivals'])/st_1), 0, 1))
    LL2 = np.sum(stats.norm.logcdf(((aux_cen['Observed_LastBid']-aux_cen['Windstream']-kappa*(aux_cen['Observed_LastBid']-aux_cen['Rivals']))/st_2).to_numpy(), 0, 1))
    LL=LL1+LL2+aux_cen.shape[0]*np.log(1/st_1)
    #aux_unc['cdf_aux'] = np.nan
    aux_unc['cdf_aux'] = stats.norm.cdf((aux_unc['Observed_LastBid']-aux_unc['Rivals']), 0, st_1).tolist()
    LL=LL+(aux_unc.loc[aux_unc['cdf_aux']<=0.001,:].shape[0])*(np.log(0.00000001)+np.log(1/(st_2)))
    
    for index, row in aux_unc.loc[aux_unc['cdf_aux']>0.001,:].iterrows():
            aux_draws=df_draws.copy()
            aux_draws['draws']=aux_draws['draws']*st_1
            aux_draws=aux_draws.loc[(aux_draws['draws']<(row['Observed_LastBid']-row['Rivals'])),:]
            aux_draws_num=aux_draws.shape[0]
            if aux_draws_num>=1:
                LL3=np.sum(stats.norm.pdf(((row['Observed_LastBid']*aux_draws['constant']-row['Windstream']*aux_draws['constant']-kappa*aux_draws['draws'])/(st_2))))
                LL=LL+np.log((LL3/aux_draws_num)*stats.norm.cdf((row['Observed_LastBid']-row['Rivals']), 0, st_1)+0.000000001)+np.log(1/(st_2))
            else:
                LL=LL+np.log(0.00000001)+np.log(1/(st_2))

    neg_LL=-LL            
    return neg_LL 

# 4. Find parameters that maximize the log-likelihood

In [108]:
pd.options.mode.chained_assignment = None  # default='warn'

In [119]:
#model = minimize(Likelihood, np.array([1,1,1,1,1,1,1,1,0]), method='Nelder-Mead',options={'disp': True})
model = optimize.fmin(Likelihood, np.array([1,1,1,1,1,1,1,1,0]))

Optimization terminated successfully.
         Current function value: 454.648841
         Iterations: 713
         Function evaluations: 1018


AttributeError: 'numpy.ndarray' object has no attribute 'x'

In [121]:
model2 = optimize.fmin(Likelihood, model)
model2

Optimization terminated successfully.
         Current function value: 445.866060
         Iterations: 660
         Function evaluations: 1034


array([0.03858931, 0.45942954, 2.01903323, 0.48508908, 0.00369532,
       0.4926276 , 2.0651525 , 0.47557011, 0.65352029])

In [123]:
np.array([mu_1, mu_1x, mu_1y,st_1,mu_2,mu_2x,mu_2d,st_2,kappa])

array([0. , 0.5, 2. , 0.5, 0. , 0.5, 2. , 0.5, 0.5])

In [117]:
Likelihood(np.array([1,1,1,1,1,1,1,1,0]))
Likelihood(model2.x)
Likelihood(np.array([mu_1, mu_1x, mu_1y,st_1,mu_2,mu_2x,mu_2d,st_2,kappa]))

450.11097168548105